In [1]:
import enlighten
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from os.path import join, exists
from sklearn.model_selection import KFold
from sklearn.mixture import BayesianGaussianMixture
from sklearn.linear_model import LinearRegression
from impyute.util import find_null
from impyute.util import checks
from impyute.util import preprocess
#from impyute.imputation.cs import mice, fast_knn

In [2]:
def mice(data, **kwargs):
    """Multivariate Imputation by Chained Equations

    Reference:
        Buuren, S. V., & Groothuis-Oudshoorn, K. (2011). Mice: Multivariate
        Imputation by Chained Equations in R. Journal of Statistical Software,
        45(3). doi:10.18637/jss.v045.i03

    Implementation follows the main idea from the paper above. Differs in
    decision of which variable to regress on (here, I choose it at random).
    Also differs in stopping criterion (here the model stops after change in
    prediction from previous prediction is less than 10%).
    
    THIS IS FROM IMPYUTE, BUT I WANT TO OPTIMIZE AND/OR PARALLELIZE IT
    CURRENTLY TAKES SEVERAL DAYS TO RUN WITH ABCD IMAGING DATA

    Parameters
    ----------
    data: numpy.ndarray
        Data to impute.

    Returns
    -------
    numpy.ndarray
        Imputed data.

    """
    null_xy = find_null(data)

    # Add a column of zeros to the index values
    null_xyv = np.append(null_xy, np.zeros((np.shape(null_xy)[0], 1)), axis=1)

    null_xyv = [[int(x), int(y), v] for x, y, v in null_xyv]
    temp = []
    cols_missing = set([y for _, y, _ in null_xyv])

    # Step 1: Simple Imputation, these are just placeholders
    for x_i, y_i, value in null_xyv:
        # Column containing nan value without the nan value
        col = data[:, [y_i]][~np.isnan(data[:, [y_i]])]

        new_value = np.mean(col)
        data[x_i][y_i] = new_value
        temp.append([x_i, y_i, new_value])
    null_xyv = temp

    # Step 5: Repeat step 2 - 4 until convergence (the 100 is arbitrary)

    converged = [False] * len(null_xyv)
    
    while all(converged):
        print(converged)
        # Step 2: Placeholders are set back to missing for one variable/column
        dependent_col = int(np.random.choice(list(cols_missing)))
        missing_xs = [int(x) for x, y, value in null_xyv if y == dependent_col]

        # Step 3: Perform linear regression using the other variables
        x_train, y_train = [], []
        for x_i in (x_i for x_i in range(len(data)) if x_i not in missing_xs):
            x_train.append(np.delete(data[x_i], dependent_col))
            y_train.append(data[x_i][dependent_col])
        model = LinearRegression()
        model.fit(x_train, y_train)

        # Step 4: Missing values for the missing variable/column are replaced
        # with predictions from our new linear regression model
        temp = []
        # For null indices with the dependent column that was randomly chosen
        for i, x_i, y_i, value in enumerate(null_xyv):
            if y_i == dependent_col:
                # Row 'x' without the nan value
                new_value = model.predict(np.delete(data[x_i], dependent_col))
                data[x_i][y_i] = new_value.reshape(1, -1)
                temp.append([x_i, y_i, new_value])
                delta = (new_value-value)/value
                if delta < 0.1:
                    converged[i] = True
        null_xyv = temp
    return data

In [3]:
sns.set(style='whitegrid', context='talk')
plt.rcParams["font.family"] = "monospace"
plt.rcParams['font.monospace'] = 'Courier New'

In [4]:
PROJ_DIR = "/Volumes/projects_herting/LABDOCS/Personnel/Katie/deltaABCD_clustering/"
DATA_DIR = "data/"
FIGS_DIR = "figures/"
OUTP_DIR = "output/"

In [5]:
df = pd.read_csv(join(PROJ_DIR, DATA_DIR, "data.csv"), index_col=0, header=0)
df.drop(list(df.filter(regex='lesion.*').columns), axis=1, inplace=True)
no_2yfu = df[df["interview_date.2_year_follow_up_y_arm_1"].isna() == True].index
df = df.drop(no_2yfu, axis=0)

In [6]:
deltasmri_complete = pd.concat([df.filter(regex='smri.*change_score'), 
                                df.filter(regex='mrisdp.*change_score')], axis=1).dropna()
deltarsfmri_complete = df.filter(regex='rsfmri.*change_score').dropna(how='any')
deltarsi_complete = df.filter(regex='dmri_rsi.*change_score').dropna()
deltadti_complete = df.filter(regex='dmri_dti.*change_score').dropna()

In [7]:
# subset by atlas for smri and rsfmri (variance) data
smri_atlas = {'cdk': [], 
              'mrisdp': [],
              #'cf12': []
             }
rsfmri_atlas = {'cdk': [],
                'cortgordon': []
               }

for atlas in smri_atlas.keys():
    smri_atlas[atlas] = list(deltasmri_complete.filter(regex=f'{atlas}.*').columns)

for atlas in rsfmri_atlas.keys():
    rsfmri_atlas[atlas] = list(deltarsfmri_complete.filter(regex=f'{atlas}.*').columns)

In [8]:
smri_scs = list(deltasmri_complete.filter(regex='.*vol_scs_.*').columns)
rsfmri_scs = list(deltarsfmri_complete.filter(regex='.*_scs_.*').columns)

In [9]:
# build data subsets for clustering
subcort = smri_scs + rsfmri_scs
cdk_columns = smri_atlas['cdk'] + rsfmri_atlas['cdk'] + list(deltarsi_complete.columns) + list(deltadti_complete.columns) + subcort
cdk_data = df.filter(cdk_columns)

dcg_columns = smri_atlas['mrisdp'] + rsfmri_atlas['cortgordon'] + list(deltarsi_complete.columns) + list(deltadti_complete.columns) + subcort
dcg_data = df.filter(dcg_columns)


In [10]:
%timeit
# let's impute some missing values, heyyyyy
cdk_data_complete = mice(cdk_data.values)
dcg_data_complete = mice(dcg_data.values)

In [11]:
imputed_cdk = pd.DataFrame(data=cdk_data_complete, 
                           columns=cdk_data.columns, 
                           index=cdk_data.index)

imputed_dcg = pd.DataFrame(data=dcg_data_complete, 
                           columns=dcg_data.columns, 
                           index=dcg_data.index)

In [14]:
imputed_cdk.describe()

,smri_vol_cdk_total.change_score,smri_area_cdk_banksstslh.change_score,smri_area_cdk_cdacatelh.change_score,smri_area_cdk_cdmdfrlh.change_score,smri_area_cdk_cuneuslh.change_score,smri_area_cdk_ehinallh.change_score,smri_area_cdk_fusiformlh.change_score,smri_area_cdk_ifpllh.change_score,smri_area_cdk_iftmlh.change_score,smri_area_cdk_ihcatelh.change_score,...,rsfmri_cor_ngd_vs_scs_vtdclh.change_score,rsfmri_cor_ngd_vs_scs_crcxrh.change_score,rsfmri_cor_ngd_vs_scs_thprh.change_score,rsfmri_cor_ngd_vs_scs_cderh.change_score,rsfmri_cor_ngd_vs_scs_ptrh.change_score,rsfmri_cor_ngd_vs_scs_plrh.change_score,rsfmri_cor_ngd_vs_scs_hprh.change_score,rsfmri_cor_ngd_vs_scs_agrh.change_score,rsfmri_cor_ngd_vs_scs_aarh.change_score,rsfmri_cor_ngd_vs_scs_vtdcrh.change_score
count,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,...,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000
mean,-0.918703,-0.340286,0.726884,0.181491,0.017468,0.555755,0.199364,-0.493266,0.292936,-0.137105,...,-0.450530,-0.693718,-1.797710,-1.045328,-6.996873,3.042624,-1.115226,-0.166640,-1.891087,1.122394
std,1.145465,2.859331,2.675971,2.824850,1.851899,5.473851,1.506527,1.995164,1.883279,2.214238,...,50.408425,46.525959,49.330894,47.711904,47.703568,49.719165,49.708265,46.018154,50.177293,49.587610
min,-15.801404,-28.871111,-28.767677,-29.224690,-33.902110,-34.011429,-22.973925,-19.592383,-23.268651,-49.000962,...,-117.120923,-132.708334,-115.447568,-127.676734,-111.575335,-127.999852,-114.108845,-118.680739,-113.539713,-129.000070
25%,-1.431812,-1.649631,-0.731940,-0.700735,-0.891214,-2.472655,-0.569409,-1.143516,-0.507647,-1.304334,...,-38.517787,-31.673869,-38.017953,-34.525677,-41.825182,-33.373539,-37.898080,-31.645472,-39.279417,-35.159233
50%,-0.852066,-0.307536,0.726884,0.235203,0.030741,0.555755,0.211334,-0.382302,0.325932,-0.137105,...,-0.450530,-0.693718,-1.797710,-1.045328,-6.996873,3.042624,-1.115226,-0.166640,-1.891087,1.122394
75%,-0.312414,1.060475,2.193853,1.149336,1.013522,3.581076,0.990187,0.297042,1.134262,1.014589,...,37.385233,31.185821,34.165021,31.887070,26.248794,40.664986,35.708836,31.130496,35.393313,37.307810
max,7.788832,19.887159,21.735221,26.461783,29.078628,42.996639,20.183486,18.090274,15.528707,28.605241,...,121.894470,114.144364,113.526623,121.511949,117.248303,129.956447,114.360318,113.762235,129.075027,131.259852


In [15]:
imputed_dcg.describe()

,mrisdp_1.change_score,mrisdp_2.change_score,mrisdp_3.change_score,mrisdp_4.change_score,mrisdp_5.change_score,mrisdp_6.change_score,mrisdp_7.change_score,mrisdp_8.change_score,mrisdp_9.change_score,mrisdp_10.change_score,...,rsfmri_cor_ngd_vs_scs_vtdclh.change_score,rsfmri_cor_ngd_vs_scs_crcxrh.change_score,rsfmri_cor_ngd_vs_scs_thprh.change_score,rsfmri_cor_ngd_vs_scs_cderh.change_score,rsfmri_cor_ngd_vs_scs_ptrh.change_score,rsfmri_cor_ngd_vs_scs_plrh.change_score,rsfmri_cor_ngd_vs_scs_hprh.change_score,rsfmri_cor_ngd_vs_scs_agrh.change_score,rsfmri_cor_ngd_vs_scs_aarh.change_score,rsfmri_cor_ngd_vs_scs_vtdcrh.change_score
count,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,...,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000
mean,-0.946806,-0.754238,-0.770600,-0.503556,-0.984857,-0.744882,-0.617957,-0.703187,-0.803155,-0.895810,...,-0.450530,-0.693718,-1.797710,-1.045328,-6.996873,3.042624,-1.115226,-0.166640,-1.891087,1.122394
std,2.345999,1.987984,2.320350,1.686890,2.788068,1.539048,1.542881,1.315405,1.499559,2.461319,...,50.408425,46.525959,49.330894,47.711904,47.703568,49.719165,49.708265,46.018154,50.177293,49.587610
min,-15.225236,-18.372330,-16.403246,-12.070811,-25.507095,-11.903757,-12.461887,-13.279857,-10.535406,-15.706603,...,-117.120923,-132.708334,-115.447568,-127.676734,-111.575335,-127.999852,-114.108845,-118.680739,-113.539713,-129.000070
25%,-2.185315,-1.692481,-1.942147,-1.326586,-2.105845,-1.579662,-1.414061,-1.441272,-1.695254,-2.307842,...,-38.517787,-31.673869,-38.017953,-34.525677,-41.825182,-33.373539,-37.898080,-31.645472,-39.279417,-35.159233
50%,-0.969075,-0.754238,-0.770600,-0.503556,-0.972889,-0.744307,-0.599689,-0.703187,-0.791327,-0.887246,...,-0.450530,-0.693718,-1.797710,-1.045328,-6.996873,3.042624,-1.115226,-0.166640,-1.891087,1.122394
75%,0.250264,0.160438,0.348410,0.287977,0.154499,0.096552,0.218694,0.053580,0.079319,0.580753,...,37.385233,31.185821,34.165021,31.887070,26.248794,40.664986,35.708836,31.130496,35.393313,37.307810
max,16.914236,15.974466,13.784666,26.066154,24.571667,9.303774,10.719937,10.718566,18.828452,22.926094,...,121.894470,114.144364,113.526623,121.511949,117.248303,129.956447,114.360318,113.762235,129.075027,131.259852


In [20]:
imputed_cdk = pd.concat([imputed_cdk, df['rel_family_id.baseline_year_1_arm_1'].astype(int)], axis=1)
imputed_dcg = pd.concat([imputed_dcg, df['rel_family_id.baseline_year_1_arm_1'].astype(int)], axis=1)

In [24]:
imputed_dcg['rel_family_id'] = imputed_dcg['rel_family_id.baseline_year_1_arm_1'].astype(int)

In [25]:
imputed_dcg.drop('rel_family_id.baseline_year_1_arm_1', axis=1, inplace=True)

In [26]:
imputed_cdk['rel_family_id'] = imputed_cdk['rel_family_id.baseline_year_1_arm_1'].astype(int)
imputed_cdk.drop('rel_family_id.baseline_year_1_arm_1', axis=1, inplace=True)

In [27]:
imputed_cdk.to_csv(join(join(PROJ_DIR, DATA_DIR, "desikankillany_MICEimputed_data.csv")))
imputed_dcg.to_csv(join(join(PROJ_DIR, DATA_DIR, "destrieux+gordon_MICEimputed_data.csv")))